In [1]:
import sys
sys.path.append('../_amigocloud')

from werkzeug.wrappers import Request, Response
from jinja2 import Template
from datetime import datetime, timedelta
import collections
from datetime import datetime

from shapely import wkb
import geopandas as gpd
import pandas as pd
import contextily as ctx
import matplotlib.pyplot as plt

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests

from amigocloud import AmigoCloud

import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [2]:
ruta = open('../unidad_local.txt')
ruta = ruta.readlines()[0]
print(ruta)
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

G:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON


## conversion de poligono

In [3]:
def convertir_wkb(wkb_data):
    return wkb.loads(wkb_data, hex=True)

## conrvertir formato fecha

In [4]:
# convierte de formato YYYY-mm-dd H:M:S+z a d/m/YYYY
def convertir_formato_fecha(fecha):
    new_formato = datetime.strptime(fecha, "%Y-%m-%d %H:%M:%S%z").strftime("%d/%m/%Y")
    return new_formato

## ejecutar query

In [5]:
# ejecuta cualquier query sql en el proyecto que se le indique
# requiere el id de proyecto, query a ejecutar y tipo solicitud (get o post)
def ejecutar_query_sql(id_project, query, tipo_sql):
    # define la url del proyecto para ejecutar el querry
    url_proyecto_sql = f'https://app.amigocloud.com/api/v1/projects/{id_project}/sql'
    # crea la estructura de query para amigocloud
    query_sql = {'query': query}
    # variable para almacenar resultado
    resultado_get = ''
    # eleige que tipo de solicitud se realizara (get o post)
    if tipo_sql == 'get': 
        resultado_get = amigocloud.get(url_proyecto_sql, query_sql)
    elif tipo_sql == 'post':
        resultado_get = amigocloud.post(url_proyecto_sql, query_sql)
    else:
        resultado_get = 'Se a seleccionado un tipo de solicitud erroneo.'
    return resultado_get

## ejecuta query segun id_query (ejecucion de queryes de un proyecto)

In [6]:
# ejecuta un query que esta almacenado en un proyecto de amigocloud (generalmente un update),
# requiere id de proyecto e id de query
# retorna cuantas filas fueron afectadas
def ejecutar_query_por_id(id_project, id_query, tipo_sql):
    # obtiene el query basado en el id_project y el id_query
    get_query = amigocloud.get(f'https://app.amigocloud.com/api/v1/projects/{id_project}/queries/{id_query}')
    # se extrae solo el texto del query
    query = get_query['query']
    # ejecuta el query_sql con metodo post y guarda la respuesta
    respuesta_post = ejecutar_query_sql(id_project, query, tipo_sql)
    # retorna el numero de filas afectadas por el query
    return respuesta_post

## convertir dic en obj

In [7]:
# convierte un dict a un obj
# recibe el dict y el nombre con el que se creara el obj
def convertir_dict_obj(diccionario, name):
    return collections.namedtuple(name, diccionario.keys())(*diccionario.values())

## crear CITE

In [8]:
def crear_cite(cod_canhero, nom_canhero, tipo_reporte):
    ruta_api_google = ruta + '\_keys\client_secret_google_sheets.json'
    # Configura las credenciales
    scope = ["https://spreadsheets.google.com/feeds",
             'https://www.googleapis.com/auth/spreadsheets', 
             "https://www.googleapis.com/auth/drive.file", 
             "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(ruta_api_google, scope)
    client = gspread.authorize(creds)
    # Abre la hoja de cálculo con ID y la hoja específica
    spreadsheet = client.open_by_key("1JROdo_Mjx6267cDfFvTFvhk1aE2_XTc28PfTgRxfwNQ")
    sheet = spreadsheet.sheet1
    # Lee los datos
    data_sheet = sheet.get_all_records()
    df = pd.DataFrame(data_sheet)
    # Encuentra el número más grande y lo incrementamos y agrega un nuevo registro
    max_value = df['No'].max()
    no = int(max_value + 1)
    # llenamos los demas campos
    fecha = datetime.now().strftime("%m/%d/%Y")
    cod_ca = int(cod_canhero)
    nom_ca = nom_canhero
    tipo = tipo_reporte
    #agregar el registro
    sheet.append_row([no, fecha, cod_ca, nom_ca, tipo])
    return no

In [9]:
# variables globales
proyecto_id = 31874

buscar_reg_nuevos = 6476
cargar_lotes_quema = 6474
calc_area_lotes = 5078
calc_total_insp = 6475

In [10]:
# revisa y extiste registros nuevos (campo reporte_generado en false)
rec_nuevos = ejecutar_query_por_id(proyecto_id, buscar_reg_nuevos, 'get')
    # se queda con la parte de darta
rec_nuevos = rec_nuevos['data']
    # extrae el id de los nuevos regitros
id_nuevos = [i['id'] for i in rec_nuevos]

In [11]:
def buscar_nuevos():
    # revisa y extiste registros nuevos (campo reporte_generado en false)
    rec_nuevos = ejecutar_query_por_id(proyecto_id, buscar_reg_nuevos, 'get')
    # se queda con la parte de darta
    rec_nuevos = rec_nuevos['data']
    # extrae el id de los nuevos regitros
    id_nuevos = [i['id'] for i in rec_nuevos]
    return id_nuevos

In [12]:
def ejecutar_scripts_sql():
    # ejecuatar scripts generales para completar campos y recalculos
    exe_cargar_lotes_quema = ejecutar_query_por_id(proyecto_id, cargar_lotes_quema, 'post')
    exe_calc_area_lotes = ejecutar_query_por_id(proyecto_id, calc_area_lotes, 'post')
    exe_calc_total_insp = ejecutar_query_por_id(proyecto_id, calc_total_insp, 'post')

In [13]:
def obtener_inspeccion(id_insp):
    # seleccionar un registro
    # crear consulta
    query = f'select * from dataset_351059 where id = {id_insp}'
    # ejecutar consulta
    inspeccion = ejecutar_query_sql(proyecto_id, query, 'get')
    # extrae la seccion de data
    inspeccion = inspeccion['data']
    # extrae el primer elemento, solo hay un elemento
    inspeccion = inspeccion[0]
    # convertion de formato de fechas
    inspeccion['date'] = convertir_formato_fecha(inspeccion['fecha_registro'])
    inspeccion['fecha_inspeccion'] = convertir_formato_fecha(inspeccion['fecha_inspeccion'])
    inspeccion['fecha_quema'] = convertir_formato_fecha(inspeccion['fecha_quema'])
    # extraccion de codigo y nombre del cañero
    cod_ca = inspeccion['canhero'].split(' / ')[0]
    nom_ca = inspeccion['canhero'].split(' / ')[1]
    # crear y asignar cite
    cite = crear_cite(cod_ca, nom_ca, 'QUEMA')
    inspeccion['cite'] = cite
    # convertir el dict en objeto
    insp = convertir_dict_obj(inspeccion, 'insp')
    return insp

In [14]:
def obtener_lotes(id_insp):
    # seleccionar todos los lotes marcados con la inspeccion
    # crear consulta
    query = f'select * from dataset_351061 where id_inspeccion = {id_insp}'
    # ejecutar consulta
    lotes = ejecutar_query_sql(proyecto_id, query, 'get')
    # extraer solo la seccion de data
    lotes = lotes['data']
    return lotes

In [15]:
# elimina todos los dic duplicados basandose en "unidad_01", y concerva solo en cop_prop y nom_prop
# con esto se obtiene un dict de propiedades de la inspeccion
def eliminar_duplicados_y_conservar_campos(lista, campo_clave, campos_a_conservar):
    vistos = set()
    nueva_lista = []
    for diccionario in lista:
        valor = diccionario[campo_clave]
        if valor not in vistos:
            vistos.add(valor)
            nuevo_diccionario = {campo: diccionario[campo] for campo in campos_a_conservar}
            nueva_lista.append(nuevo_diccionario)
    return nueva_lista

In [16]:
def propiedades_lotes(props):
    # recorrer las propiedades, y agregar los lotes correspondientes
    # se crea una lista de objetos propiedad con los respectivos lotes agregados a cada propiedad
    propiedades = []
    for prop in props:
        prop['lote'] = []
        lotes_select = [lote for lote in lotes if lote['unidad_01'] == prop['unidad_01']]
        for lote_select in lotes_select:
            lote = convertir_dict_obj(lote_select, 'lote')
            prop['lote'].append(lote)
        propiedades.append(convertir_dict_obj(prop, 'propiedad'))
    return propiedades

In [17]:
def obtener_fotos(insp_amigo_id):
    # buscar todas las fotos que son parte de la inspeccion
    # crear consulta
    query = f'select s3_filename from gallery_61142 where source_amigo_id = \'{insp_amigo_id}\''
    # ejecutar consulta
    fotos = ejecutar_query_sql(proyecto_id, query, 'get')
    # extrae la seccion de data
    fotos = fotos['data']
    return fotos

In [18]:
def generar_planos(insp, propiedades):
    # generar planos
    lista_planos = []
    path = ''
    for propiedad in propiedades:
        lotes_lista = []
        for lote in propiedad.lote:
            lotes_lista.append(lote._asdict())
        df = pd.DataFrame(lotes_lista)
        df['geometria'] = df['geometria'].apply(convertir_wkb)

        #Convertir a GeoDataFrame
        data = gpd.GeoDataFrame(df, geometry='geometria')

        data['coords'] = data['geometria'].apply(lambda x: x.representative_point().coords[:])
        data['coords'] = [coords[0] for coords in data['coords']]

        data.crs = "EPSG:4326"
        data = data.to_crs(epsg=3857)
        
        fig = plt.figure(i, figsize=(20,20))
        ax = None
        ax = fig.add_subplot()

        data.apply(lambda x: ax.annotate(text=x.unidad_05 + ' \n' + str(x.area) + ' ha', xy=x.geometria.centroid.coords[0], ha='center', va='center', color='black', fontsize=12, weight=1000, bbox=dict(facecolor=(1,1,1,0.3), edgecolor='none', pad=0)), axis=1);
    
        minx, miny, maxx, maxy = data.total_bounds
        ax.set_xlim(minx - 500, maxx + 500)
        ax.set_ylim(miny - 400, maxy + 400)

        data.plot(ax=ax, edgecolor='r', facecolor=(0,0,0,0), linewidth=2, figsize=(20,20))
    
        ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery)
        ax.set_axis_off()
        ax.set_title(str(propiedad.unidad_01) + ' / ' + str(propiedad.unidad_02), fontsize=20)
        path = ruta + '/planos/' + str(insp.amigo_id) + '_' + str(propiedad.unidad_01) + '.jpeg'
        lista_planos.append(path)
        fig.savefig(path, dpi = 300, bbox_inches='tight')
        plt.clf()
    return lista_planos

In [19]:
def generar_reporte(insp, propiedades, fotos, lista_planos):
    # generar reporte
    # asignacion de template
    doc = DocxTemplate(ruta + "/templates/tpl_infome_quema.docx")

    #generar lista de InlineImage de planos 
    lista_InlineImage = []
    for plano in lista_planos:
        lista_InlineImage.append(docxtpl.InlineImage(doc, image_descriptor=plano, width=Mm(150)))

    #descargar fotos y generar lista InlineImage
    lista_fotos_inline = []
    for foto in fotos:
        response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
        file = open(ruta + '/fotos/' + foto['s3_filename'], "wb")
        file.write(response.content)
        file.close()
        lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor= ruta + '/fotos/' + foto['s3_filename'], width=Mm(120))})

    firma_respon = None
    if insp.responsable_tec == 'Rogelio Acuña Rodríguez':
        firma_respon = docxtpl.InlineImage(doc, image_descriptor=ruta + '/templates/firma_rogelio.png', width=Mm(60))
    else:
        firma_respon = docxtpl.InlineImage(doc, image_descriptor=ruta + '/templates/firma_jaldin.png', width=Mm(60))

    context = {'insp':insp, 'propiedades':propiedades, 'planos':lista_InlineImage, 'fotos':lista_fotos_inline, 'firma':firma_respon}

    doc.render(context)

    # formato de nombre de archivo: "123_CQ_01-01-2022_NOMBRE"
    cod_nom = insp.canhero.split(' / ')
    file_name = cod_nom[0] + '_IDCQ_' + insp.fecha_inspeccion.replace('/','-') + '_' + cod_nom[1] + '_' + str(insp.id)

    doc.save(ruta + '/informes/_' + file_name + '.docx')

In [20]:
def cambiar_estado_informe(id_insp):
    # actualizar estado de informe_generado a true
    # crear consulta
    query = f'update dataset_351059 set informe_generado = true where id = {id_insp}'
    # ejecutar consulta
    res = ejecutar_query_sql(proyecto_id, query, 'post')
    return res

In [21]:
reg_nuevos = buscar_nuevos()

In [22]:
reg_nuevos

[212]

In [23]:
ins = obtener_inspeccion(212)

In [24]:
ins

insp(id=212, informe_generado=False, fecha_registro='2025-06-27 19:32:12+00:00', fecha_quema='25/06/2025', fecha_inspeccion='27/06/2025', canhero='15928 / YUPANQUI CONDORI EDSON', superficie_total=None, rendimiento=70.0, produccion=None, cite=478, inicio_incendio='Fuera de la propiedad', causa='Quema de barbecho', responsable_de_quema='Quema de barbecho en propiedad vecina', observaciones=None, link_informe=None, tipo_cosecha='MECANIZADO', amigo_id='341c0eb30de045e4bf898e9f96686e9c', responsable_tec='Bismark Socompi Rodríguez', ubicacion='0104000020E610000001000000010100000029CDE671189A4FC0E474482014FC30C0', date='27/06/2025')

In [29]:
exe_cargar_lotes_quema = ejecutar_query_por_id(proyecto_id, cargar_lotes_quema, 'post')

In [36]:
exe_cargar_lotes_quema

{'query': 'INSERT INTO dataset_351061 (unidad_01, unidad_02, unidad_05, id_inspeccion, geometria)\r\nSELECT \r\n    c.unidad_01,\r\n    c.unidad_02,\r\n    c.unidad_05,\r\n    i.id AS id_inspeccion,\r\n    c.wkb_geometry AS geometria\r\nFROM \r\n    dataset_351059 i\r\nJOIN \r\n    dataset_375429 c \r\nON \r\n    ST_Intersects(i.ubicacion, c.wkb_geometry)\r\nWHERE NOT EXISTS (\r\n    SELECT 1 \r\n    FROM dataset_351061 l \r\n    WHERE l.id_inspeccion = i.id \r\n);',
 'count': 0}

In [35]:
lotes = obtener_lotes(i)

NameError: name 'i' is not defined

In [28]:
props = eliminar_duplicados_y_conservar_campos(lotes, 'unidad_01', ['unidad_01', 'unidad_02'])
props

NameError: name 'lotes' is not defined

In [31]:
propiedades = propiedades_lotes(props)
propiedades

NameError: name 'props' is not defined

In [37]:
fotos = obtener_fotos(insp.amigo_id)
fotos

NameError: name 'insp' is not defined

In [38]:
lista_planos = generar_planos(insp, propiedades)

NameError: name 'insp' is not defined

In [ ]:
while True:
    reg_nuevos = buscar_nuevos()
    if len(reg_nuevos) == 0:
        print('No se encontraron registros nuevos')
        continue
    for i in reg_nuevos:
        insp = obtener_inspeccion(i)
        ejecutar_scripts_sql()
        lotes = obtener_lotes(i)
        if len(lotes) == 0:
            print(f'Inspeccion {i} no tiene lotes asignados')
            continue
        # de lotes eliminar todos los duplicados, y solo se queda con el codigo y nombre de propiedad, esto sera el objeto de propiedades que son parte de la inspeccion
        props = eliminar_duplicados_y_conservar_campos(lotes, 'unidad_01', ['unidad_01', 'unidad_02'])
        propiedades = propiedades_lotes(props)
        fotos = obtener_fotos(insp.amigo_id)
        if len(fotos) == 0:
            print(f'Inspeccion {i} no tiene fotos')
        lista_planos = generar_planos(insp, propiedades)
        print(insp)
        print(propiedades)
        print(fotos)
        generar_reporte(insp, propiedades, fotos, lista_planos)
        cambiar_estado_informe(i)
        print(f'Informe generado de {insp.canhero}')

insp(id=212, informe_generado=False, fecha_registro='2025-06-27 19:32:12+00:00', fecha_quema='25/06/2025', fecha_inspeccion='27/06/2025', canhero='15928 / YUPANQUI CONDORI EDSON', superficie_total=4.24, rendimiento=70.0, produccion=296.8, cite=480, inicio_incendio='Fuera de la propiedad', causa='Quema de barbecho', responsable_de_quema='Quema de barbecho en propiedad vecina', observaciones=None, link_informe=None, tipo_cosecha='MECANIZADO', amigo_id='341c0eb30de045e4bf898e9f96686e9c', responsable_tec='Bismark Socompi Rodríguez', ubicacion='0104000020E610000001000000010100000029CDE671189A4FC0E474482014FC30C0', date='27/06/2025')
[propiedad(unidad_01='1601', unidad_02='PAZ ESTENSORO II--YUPANQUI EDSON', lote=[lote(fecha_registro='2025-06-27 19:41:50.874681+00:00', amigo_id='ecdccb7556c9475c8762287d3345b190', id=87, geometria='0106000020E6100000010000000103000000010000000800000081EF91280F9A4FC05766005E55FC30C0E1A41FC3199A4FC072A609DB4FFC30C090D552E5209A4FC0B2C8659247FC30C0CC1363F4379A4FC0